# 量子テレポーテーション


Kifumi Numata (Aug 17, 2023)

In [ ]:
# Qiskitライブラリーを導入
from qiskit import *
from qiskit.visualization import plot_histogram, plot_bloch_multivector

import numpy as np

In [ ]:
# 3量子ビット回路を用意
qc = QuantumCircuit(3,3)    

# Aliceのもつ暗号の量子状態ψを今回はRxで作ります
qc.rx(np.pi/3,0)
qc.barrier()    #回路を見やすくするために入れます

# 回路を描画
qc.draw(output="mpl")

In [ ]:
backend = Aer.get_backend('statevector_simulator')
out_vector = backend.run(qc,shots=1).result().get_statevector() # set shots = 1

plot_bloch_multivector(out_vector)

In [ ]:
# 量子もつれを作ります
qc.h(1)
qc.cx(1, 2)
qc.barrier()    #回路を見やすくするために入れます

# 回路を描画
qc.draw(output="mpl")

In [ ]:
# AliceがCNOTとHで、自分の量子ビット2つをエンタングルさせ測定します。
qc.cx(0, 1)
qc.h(0)
qc.barrier()
qc.measure(0, 0)
qc.measure(1, 1)

# 回路を描画
qc.draw(output="mpl")

In [ ]:
#Aliceが測定結果をBobに送り、Bobが結果に合わせて操作します
with qc.if_test((0, 1)): # 古典レジスター0の値が1だったらZゲートをq2にかける
    qc.z(2)
with qc.if_test((1, 1)): # 古典レジスター1の値が1だったらXゲートをq2にかける
    qc.x(2)
    
qc.draw(output="mpl")

量子テレポーテーション回路が完成しました。qubit2にqubit0の量子状態($R_x(\pi/3)$)が転送されていることを確認しましょう。

In [ ]:
backend = Aer.get_backend('statevector_simulator')
out_vector = backend.run(qc,shots=1).result().get_statevector()

plot_bloch_multivector(out_vector)

Bobに暗号の量子状態が転送されたことを確認するために、最後にBobの量子ビットに逆向きのX軸回転を適用して、QASMシミュレーターで実験し、測定結果が0になることを確認します。

In [ ]:
# 3量子ビット回路を用意
qc = QuantumCircuit(3,3)  

# Aliceのもつ未知の量子状態ψをRxで作ります。角度はπ/3にしました。
qc.rx(np.pi/3,0)
qc.barrier()    #回路を見やすくするために入れます

# 量子もつれを作ります
qc.h(1)
qc.cx(1, 2)
qc.barrier()

# AliceがCNOTとHで自分の量子ビット2つをエンタングルさせ測定します。
qc.cx(0, 1)
qc.h(0)
qc.barrier()
qc.measure(0, 0)
qc.measure(1, 1)

#Aliceが測定結果をBobに送り、Bobが結果に合わせて操作します
with qc.if_test((0, 1)): # 古典レジスター0の値が1だったらZゲートをq2にかける
    qc.z(2)
with qc.if_test((1, 1)): # 古典レジスター1の値が1だったらXゲートをq2にかける
    qc.x(2)

# 未知の量子状態ψの逆ゲートをかけて０が測定できるか確かめます
qc.rx(-np.pi/3, 2)    
qc.measure(2, 2)

qc.draw(output="mpl")

In [ ]:
# QASMシミュレーターで実験
backend = Aer.get_backend('qasm_simulator')
job_sim = backend.run(qc)
result_sim = job_sim.result()

#  測定された回数を表示
counts = result_sim.get_counts()
print(counts)

# ヒストグラムで測定された確率をプロット
plot_histogram(counts)

qiskitのビット配列は右端が0なので、Bobのビット(qubit 2)は左端です。左端のビットが全て0になっていることが確認できましたか？

## 実デバイスでの実行

実デバイスで上記の実験を行ってみましょう。（量子計算の途中の測定結果を使ってさらに計算を続ける「動的回路」は、昨年末に実装されたばかりの新機能です！）

In [ ]:
# アカウント情報をロードして、使える量子デバイスを確認します
from qiskit_ibm_provider import IBMProvider

# provider = IBMProvider(instance='ibm-q/open/main')  # Openプロバイダーの場合
provider = IBMProvider(instance='ibm-q-education/ibm-3/kawasaki-camp')
provider.backends()

In [ ]:
#一番空いているバックエンドを自動的に選択します
from qiskit_ibm_provider import least_busy

small_devices = provider.backends(simulator=False, operational=True, dynamic_circuits=True, min_num_qubits=3)
real_backend = least_busy(small_devices)
print("最も空いているバックエンドは: ", real_backend.name)

In [ ]:
# 以下でバックエンドを直接設定することもできます
real_backend = provider.get_backend('ibm_lagos')

In [ ]:
# 実機のバックエンドでの実行に最適な回路に変換します
qc_compiled = transpile(qc, real_backend)
# 動的回路「dynamic=True」を入れて実行します
job = real_backend.run(qc_compiled, shots=1024, dynamic=True)

job_id = job.job_id() # 実行に時間がかかるのでjob_idを表示します
print(job_id)

In [ ]:
job_default = provider.backend.retrieve_job('cjbjesslik6102r6cpg0') # job_idの例です
job_default.status() # ジョブの実行状態を確認します

`job has successfully run` となったら、以下で結果を確認します。

In [ ]:
# 結果を確認します
exp_result = job_default.result()
exp_counts = exp_result.get_counts()
print(exp_counts)
plot_histogram(exp_counts)

Qiskitのビット配列は右端が0なので、Bobのビット(qubit 2)は左端です。Bobのビットの結果のみ抜き出します。

In [ ]:
from qiskit.result import marginal_counts
bobs_qubit = 2
bobs_counts = marginal_counts(exp_counts, [bobs_qubit])
plot_histogram(bobs_counts)

## 演習

量子テレポーテーション回路を自分で組んで表示してみましょう。<br>
今回は、未知の量子状態をY軸回転ゲート(`ry`を使います。角度は自由に決めます。)で作ってみましょう。<br>
Bobに未知の量子状態が転送されたことを確認するために、最後にBobの量子ビットに逆向きのY軸回転(角度にマイナスをつけます)を適用して、QASMシミュレーターで実験し、測定結果が0になることを確認しましょう。

時間に余裕がある方は、実デバイスでも上記の実験を行ってみましょう。

In [ ]:
# 3量子ビット回路を用意
qc = QuantumCircuit(3,3)    

##量子テレポーテーションのコードを作ってください##
# Aliceのもつ未知の量子状態ψをRyで作ります。


# 量子もつれを作ります


# AliceがCNOTとHでψと自分のEPRペアをエンタングルさせ測定します。


#Aliceが測定結果をBobに送り、Bobが結果に合わせて操作します


# 未知の量子状態ψの逆向きの演算をかけて０が測定できるか確かめます


# 回路を描画
qc.draw(output="mpl")

In [ ]:
# QASMシミュレーターで実験の際に使うコード例
backend = Aer.get_backend('qasm_simulator')
job_sim = backend.run(qc)
result_sim = job_sim.result()

#  測定された回数を表示
counts = result_sim.get_counts()
print(counts)

# ヒストグラムで測定された確率をプロット
plot_histogram(counts)

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table